# Skytrax Web Scrape
* beautiful soup
* 參考網站

# Install & Imports

In [153]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime

In [8]:
# # get webpage
# url = 'https://www.airlinequality.com/review-pages/latest-airline-reviews/'
# #headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
# response = requests.get(url)
# print(response.text)

# # optional
# print(response.content)
# print(response.encoding)
# print(response.raw.read(50))

In [89]:
# 各航空的 url
url = 'https://www.airlinequality.com/review-pages/a-z-airline-reviews/'
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

review_url_list =[]

# tab content
tabs = soup.findAll('div',{'class' :'tabs-content'})
tab = tabs[0].findAll('div', {'class' : 'content'})

if len(tab)>0:    
    for idx, one in enumerate(tab):    
        print('tab No.', idx+1)
        for a in one.find_all('a', href=True):
            review_url_list.append(tmp)

# 共543 個網址


tab No. 1

tab No. 2

tab No. 3

tab No. 4

tab No. 5

tab No. 6

tab No. 7

tab No. 8

tab No. 9

tab No. 10

tab No. 11

tab No. 12

tab No. 13

tab No. 14

tab No. 15

tab No. 16

tab No. 17

tab No. 18

tab No. 19

tab No. 20

tab No. 21

tab No. 22

tab No. 23

tab No. 24

tab No. 25

tab No. 26


In [91]:
# len(review_url_list)
# 543

In [149]:
# use beautiful soup to parse reviews
# 直接找呈現100則評論的頁面
# 內容網址 : 'https://www.airlinequality.com/airline-reviews/aerolineas-argentinas'
for url in review_url_list[:1]:
    print(url, '\n')
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    allarticles = soup.findAll('article')

    articles = allarticles[0].findAll('article')
    print('Number of reviews: ', len(articles))
    if len(articles)>0:    
        for idx, one in enumerate(articles):    
        #     print('\nReview No.', idx+1)
        #     #print(one) # If we want to look at the complete section source
            
            # print('by:',one.find('span',{'itemprop': 'name'}).text)
            # print('Published:',one.find('time',{'itemprop': 'datePublished'}).text)
            # print('Ratings:',one.find('span',{'itemprop': 'ratingValue'}).text + '/' +one.find('span',{'itemprop': 'bestRating'}).text)
            # print('Header:',one.find('h2',{'class': 'text_header'}).text )
            # print(one.find('div',{'class': 'text_content'}).text.split('|')[1])

            data_cnt =0
            star_cnt =0

            for element in one.find_all('tr'):
                print(element.find_all('td')[0]['class'][1])
                
                if not (element.find_all('td')[1].text).isnumeric():
                    print('the text : ' , element.find_all('td')[1].text)
                    data_cnt+=1
                else:
                    stars = len(element.find_all('span',{'class': 'star fill'}))
                    star_cnt+=1
                    print(stars)

                # if stars == 0:
                #     rating_value = (element.find_all('td')[1]).text
                #     record[rating_element] = rating_value.strip()
                # else:
                #     record[rating_element] = int(stars)

            print(data_cnt , star_cnt)
            


https://www.airlinequality.com/airline-reviews/azur-air-germany?sortby=post_date%3ADesc&pagesize=100 

Number of reviews:  2
aircraft
the text :  Boeing 767-300
type_of_traveller
the text :  Solo Leisure
cabin_flown
the text :  Economy Class
route
the text :  Düsseldorf to Palma
date_flown
the text :  June 2018
seat_comfort
1
cabin_staff_service
1
food_and_beverages
1
inflight_entertainment
1
ground_service
1
wifi_and_connectivity
1
value_for_money
1
recommended
the text :  no
6 7
aircraft
the text :  Boeing 737-900
type_of_traveller
the text :  Solo Leisure
cabin_flown
the text :  Economy Class
route
the text :  Palma to Dusseldorf
date_flown
the text :  July 2018
seat_comfort
1
cabin_staff_service
2
food_and_beverages
2
ground_service
1
value_for_money
1
recommended
the text :  no
6 5


In [140]:
# 'https://www.airlinequality.com/airline-reviews/azur-air-germany?sortby=post_date%3ADesc&pagesize=100'
tmp = review_url_list[0]

import re
regex = re.compile(r'airline-reviews/(\S*)\?sortby=post_date%3ADesc&pagesize=100')
match = regex.search(tmp)
print(match.group(1))

azur-air-germany


In [154]:
# 匯出成csv
for idx , url in enumerate(review_url_list[:1]):

    print(idx)
    print(url)
    # get airport name
    regex = re.compile(r'airline-reviews/(\S*)\?sortby=post_date%3ADesc&pagesize=100')
    match = regex.search(url)
    airportname = match.group(1)

    # get response
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    data = []
    allarticles = soup.findAll('article')
    articles = allarticles[0].findAll('article')


    
    for one in articles:
        record = {}
        record['airport'] = airportname
        
        overallRating = one.find('span',{'itemprop': 'ratingValue'}).text
        if str(overallRating) == 'None':
            overallRating = 0
        else:
            record['ratingValue'] = overallRating       
        
        record['author'] = one.find('span',{'itemprop': 'name'}).text.strip()

        tempdate = one.find('time',{'itemprop': 'datePublished'}).text
        tempdate = re.sub(r'(\d)(st|nd|rd|th)', r'\1', tempdate)
        date_time_obj = datetime.strptime(tempdate, '%d %B %Y')
        record['datePublished'] = date_time_obj

        tmptext = one.find('div',{'class': 'text_content'}).text
        text = tmptext.split('|')
        if len(text)>1:
            textindex=1
            verifyindex=0
        else:
            textindex=0
            verifyindex=-1
        record['reviewText'] = text[textindex].strip()

        if verifyindex!=-1:
            if 'Not' in tmptext.split('|')[verifyindex]:
                record['verified'] = False
            else:
                record['verified'] = True
        else:
            record['verified'] = False
        
        ## 其他細項
        for element in one.find_all('tr'):

            rating_element = element.find_all('td')[0]['class'][1]
            if not (element.find_all('td')[1].text).isnumeric():
                rating_value = (element.find_all('td')[1]).text
                record[rating_element] = rating_value.strip()
            else:
                stars = len(element.find_all('span',{'class': 'star fill'}))
                record[rating_element] = int(stars)

        data.append(record)


0
https://www.airlinequality.com/airline-reviews/azur-air-germany?sortby=post_date%3ADesc&pagesize=100


In [156]:
df = pd.DataFrame(data)

In [157]:
df

,airport,ratingValue,author,datePublished,reviewText,verified,aircraft,type_of_traveller,cabin_flown,route,date_flown,seat_comfort,cabin_staff_service,food_and_beverages,inflight_entertainment,ground_service,wifi_and_connectivity,value_for_money,recommended
0,azur-air-germany,1,H Johnson,2018-07-15,Düsseldorf to Palma. Azur Air Germany operates...,True,Boeing 767-300,Solo Leisure,Economy Class,Düsseldorf to Palma,June 2018,1,1,1,1.0,1,1.0,1,no
1,azur-air-germany,1,H Meisteran,2018-07-11,Palma to Dusseldorfv. Azur Air Germany is a ne...,True,Boeing 737-900,Solo Leisure,Economy Class,Palma to Dusseldorf,July 2018,1,2,2,NaN,1,NaN,1,no
